## Arken  - Wikidata
version 1.1

WD egenskap [Property:P8899](https://www.wikidata.org/wiki/Property:P8899) 
* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/Arken.ipynb)  
* Task [T269064](https://phabricator.wikimedia.org/T269064)
----


#### Other sources we sync
* [Arken](https://github.com/salgo60/open-data-examples/blob/master/Arken.ipynb) 
  * WD [Property:P8899](https://www.wikidata.org/wiki/Property:P8899) 
* [Kulturpersoner Uppsalakyrkogård](https://github.com/salgo60/open-data-examples/blob/master/Check%20WD%20kulturpersoner%20uppsalakyrkogardar.ipynb)
* [Litteraturbanken](https://github.com/salgo60/open-data-examples/blob/master/Litteraturbanken%20Author.ipynb) 
  * WD property [P5101](https://www.wikidata.org/wiki/Property_talk:P5101) [P5123](https://www.wikidata.org/wiki/Property_talk:P5123)
* [Nobelprize.org](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)
  * WD [property 8024](https://www.wikidata.org/wiki/Property:P8024)
* [SBL](https://github.com/salgo60/open-data-examples/blob/master/SBL.ipynb) 
  * WD [property 3217](https://www.wikidata.org/wiki/Property:P3217) 
* [SKBL](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb)
  * WD [property 4963](https://www.wikidata.org/wiki/Property:P4963)
* [Svenska Akademien](https://github.com/salgo60/open-data-examples/blob/master/Svenska%20Akademien.ipynb) 
  * WD [property 5325](https://www.wikidata.org/wiki/Property:P5325) 


In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", now)

Last run:  2021-02-13 06:43:22.151444


In [2]:
import urllib3, json
import pandas as pd   
from bs4 import BeautifulSoup
import sys
import pprint
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import trange  
from wikidataintegrator import wdi_core, wdi_login

endpoint_url = "https://query.wikidata.org/sparql"

SparqlQuery = """SELECT ?item ?arkid WHERE {
?item wdt:P8899 ?arkid
}"""


http = urllib3.PoolManager()

# Query https://w.wiki/Vo5
def get_results(endpoint_url, query):
    user_agent = "user  salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

SparQlResults = get_results(endpoint_url, SparqlQuery)
length = len (SparQlResults["results"]["bindings"])
dfWikidata = pd.DataFrame(columns=['WD', 'arkid'])
    
for r in trange(0,length):
    resultSparql = SparQlResults["results"]["bindings"][r]
    wd = resultSparql["item"]["value"].replace("http://www.wikidata.org/entity/","") 
    try: 
        wdArkid= resultSparql["arkid"]["value"] 
    except:
        wdArkid = ""    
    dfWikidata = dfWikidata.append({'WD': wd, 'arkid': wdArkid}, ignore_index=True)
  

In [3]:
dfWikidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   WD      3372 non-null   object
 1   arkid   3372 non-null   object
dtypes: object(2)
memory usage: 52.8+ KB


In [4]:
dfWikidata.head(200)

,WD,arkid
0,Q890742,"Anna,-prinsessa-av-Sverige"
1,Q1698723,aderne-john
2,Q1254,"Annan,-Kofi-A"
3,Q179025,"Anouilh,-Jean"
4,Q5557877,"Antoni,-Nils"
...,...,...
195,Q5695637,carlberg-b-w
196,Q793616,"prins-Carl,-hertig-av-Västergötland"
197,Q5602474,carlberg-gosta
198,Q5602592,carleson-carl


In [5]:
import urllib.parse
urlbase = "https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page="
urlbase_entry = "https://arken.kb.se"
dfArken = pd.DataFrame(columns=['nameAuth', 'urlAuth', 'Auktoriserad', 'Datum', 'Auktoritetspost'])

#for i in range(1,80):
for i in range(1,90):
    url = urlbase + str(i)
    print(url)
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, "html.parser")
    for link in soup.select('div.search-result-description a[href]'):
        nameAuth = link.string
        urlAuth = urllib.parse.unquote(link['href'].split("/")[1])
        #print ("\t",urlAuth, nameAuth)    
        urlentry = urlbase_entry + link['href']
        #print ("\t\t",urlentry)
        try:
            r_entry = http.request('GET', urlentry)
            soup_entry = BeautifulSoup(r_entry.data, "html.parser")
            Auktoriserad = ""
            Datum = ""
            Auktoritetspost = ""
            fields = soup_entry.select('div.field')
            for f in fields:
                h3 = f.select("h3")
                divText = f.select("div")
                if len(h3) > 0:
                    if "Auktoriserad" in h3[0].getText():
                        #print("\t\tAuktoriserad: " + divText[0].getText().strip())
                        Auktoriserad = divText[0].getText().strip()
                    if "Datum för verksamhetstid" in h3[0].text:
                        #print("\t\t\tDatum: " + divText[0].getText().strip())
                        Datum =  divText[0].getText().strip()
                    if "Auktoritetspost" in h3[0].text:
                        Auktoritetspost =   divText[0].getText().strip()
                        #print("\t\t\tAuktoritetspost: " + divText[0].getText().strip())
                
            dfArken = dfArken.append({'nameAuth': nameAuth, 'urlAuth': urlAuth, 'Auktoriserad': Auktoriserad, 
                              'Datum': Datum, 'Auktoritetspost': Auktoritetspost}, ignore_index=True)
  
        except:
            print("Error")
                
 

https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=1
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=2
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=3
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=4
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=5
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=6
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=7
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=8
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=9
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=10
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=11
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=12
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=13
https://arken.kb.se/actor/browse?sort=alphabetic&sortDir=asc&page=14
https://arken.kb.se/actor/browse?sort=alpha

In [6]:
dfArken.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6034 entries, 0 to 6033
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nameAuth         6034 non-null   object
 1   urlAuth          6034 non-null   object
 2   Auktoriserad     6034 non-null   object
 3   Datum            6034 non-null   object
 4   Auktoritetspost  6034 non-null   object
dtypes: object(5)
memory usage: 235.8+ KB


In [7]:
dfArken.head(200)

,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost
0,"Abenius, Margit","Abenius,-Margit","Abenius, Margit",1899-1970,https://libris.kb.se/ljx00mt45v0dfx5#it
1,"Abenius, Vera","Abenius,-Vera","Abenius, Vera",1890-1967,ediffah:kb:636923:1147851925
2,"Aber, Erich","Aber,-Erich","Aber, Erich",1904-1995,ediffah:kb:294903:1160049953
3,"Abildgaard, Nicolai","Abildgaard,-Nicolai","Abildgaard, Nicolai",1743-1809,https://libris.kb.se/sq4671cb16gj9q4#it
4,"Abrahamson, August","Abrahamson,-August","Abrahamson, August",1817-1898,https://libris.kb.se/wt7bkc9f1h1tt4z#it
...,...,...,...,...,...
195,"Andersson, George",andersson-george,"Andersson, George",,
196,"Andersson, Gunder","Andersson,-Gunder","Andersson, Gunder",1943-,https://libris.kb.se/vs688w3d219c01z#it
197,"Andersson, Gunnar",andersson-gunnar,"Andersson, Gunnar",1925-1995,
198,"Andersson, Ingeborg","Andersson,-Ingeborg","Andersson, Ingeborg",,


In [8]:
dfArken.to_csv(r'Arken.csv')

## Check diff Wikidata
* dfArken
* dfWikidata
* see also 

In [9]:
dfWikidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   WD      3372 non-null   object
 1   arkid   3372 non-null   object
dtypes: object(2)
memory usage: 52.8+ KB


In [10]:
dfArken.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6034 entries, 0 to 6033
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nameAuth         6034 non-null   object
 1   urlAuth          6034 non-null   object
 2   Auktoriserad     6034 non-null   object
 3   Datum            6034 non-null   object
 4   Auktoritetspost  6034 non-null   object
dtypes: object(5)
memory usage: 235.8+ KB


In [11]:
dfArken.sample()

,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost
5019,"Strindberg, Hans","Strindberg,-Hans","Strindberg, Hans",1884-1917,ediffah:kb:196543:1357722285


In [12]:
# Merge plotPublishedAuthor WDSKBLtot  
mergeArkenWD = pd.merge(dfWikidata, dfArken,how='outer', left_on='arkid',right_on='urlAuth',indicator=True)   
mergeArkenWD.rename(columns={"_merge": "WD_Arken"},inplace = True)
mergeArkenWD['WD_Arken'] = mergeArkenWD['WD_Arken'].str.replace('left_only','WD_only').str.replace('right_only','Arken_only')
mergeArkenWD["WD_Arken"].value_counts()  


both          3316
Arken_only    2723
WD_only         56
Name: WD_Arken, dtype: int64

In [13]:
Arken_only = mergeArkenWD[mergeArkenWD["WD_Arken"] == "Arken_only"].copy() 
WD_only = mergeArkenWD[mergeArkenWD["WD_Arken"] == "WD_only"].copy() 
# could be places etc....
WD_only.head(10)

,WD,arkid,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost,WD_Arken
1002,Q879471,Bjurholm,NaN,NaN,NaN,NaN,NaN,WD_only
1004,Q142,Frankrike,NaN,NaN,NaN,NaN,NaN,WD_only
1010,Q926728,Bromma,NaN,NaN,NaN,NaN,NaN,WD_only
1014,Q3388444,Holte,NaN,NaN,NaN,NaN,NaN,WD_only
1022,Q665230,Mondsee,NaN,NaN,NaN,NaN,NaN,WD_only
1029,Q1157266,"Edqvist, Dagmar",NaN,NaN,NaN,NaN,NaN,WD_only
1030,Q184719,antroposofi,NaN,NaN,NaN,NaN,NaN,WD_only
1036,Q10427902,SE-S-HS-Acc2012-107,NaN,NaN,NaN,NaN,NaN,WD_only
1902,Q28287,Ystad,NaN,NaN,NaN,NaN,NaN,WD_only
1904,Q64694,Dornach,NaN,NaN,NaN,NaN,NaN,WD_only


In [14]:
pd.set_option('display.max_rows', None)  
Arken_only.sample(100)

,WD,arkid,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost,WD_Arken
5728,NaN,NaN,"Södergren, Edvard","Södergren,-Edvard,-läkare","Södergren, Edvard",,,Arken_only
3601,NaN,NaN,"Berlin, Hugo",berlin-hugo,"Berlin, Hugo",1875-1963,,Arken_only
3791,NaN,NaN,Centrum för lättläst,stiftelsen-centrum-for-lattlast,Centrum för lättläst,,https://libris.kb.se/sq464f5b39vbnrz#it,Arken_only
5440,NaN,NaN,"Schlyter-Kirkham, Margaretha","Schlyter-Kirkham,-Margaretha","Schlyter-Kirkham, Margaretha",,,Arken_only
4125,NaN,NaN,Frimurarorden,Frimurarorden,Frimurarorden,,https://libris.kb.se/64jmq1kq0l8fmn6#it,Arken_only
3649,NaN,NaN,"Blomdahl, Rune","Blomdahl,-Rune","Blomdahl, Rune",,,Arken_only
5834,NaN,NaN,"Uhr, Carin","Uhr,-Carin","Uhr, Carin",1783-1830,ediffah:kb:898014:1367491819,Arken_only
5230,NaN,NaN,"Petre, Jeanne","Petre,-Jeanne","Petre, Jeanne",,,Arken_only
5712,NaN,NaN,"Sylvander, Wolmar Henrik","Sylvander,-Wolmar-Henrik","Sylvander, Wolmar Henrik",1813-1882,https://libris.kb.se/rp36f6t907brt17#it,Arken_only
3493,NaN,NaN,Artizans syndicate,Artizans-syndicate,Artizans syndicate,,,Arken_only


In [15]:
mergewithLibris = Arken_only[Arken_only["Auktoritetspost"].notnull()].copy() 
mergewithLibris

,WD,arkid,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost,WD_Arken
3372,NaN,NaN,"Abrahamsson, Maggie","Abrahamsson,-Maggie","Abrahamsson, Maggie",,,Arken_only
3373,NaN,NaN,"Abramson, August",abramson-august,"Abramson, August",,,Arken_only
3374,NaN,NaN,"Acciaioli, Benedetto Zanobi",acciaioli-benedetto-zanobi,"Acciaioli, Benedetto Zanobi",,,Arken_only
3375,NaN,NaN,"Ackeson, Dean","Ackeson,-Dean","Ackeson, Dean",,,Arken_only
3376,NaN,NaN,Adams-Ray (släkt),Adams-Ray-släkt,Adams-Ray (släkt),,ediffah:kb:688055:1137069059,Arken_only
3377,NaN,NaN,"Adams-Ray, Dagmar","Adams-Ray,-Dagmar","Adams-Ray, Dagmar",,,Arken_only
3378,NaN,NaN,Adelsköld (släkt),Adelsköld-släkt,Adelsköld (släkt),,ediffah:kb:482202:1457702780,Arken_only
3379,NaN,NaN,"Adelsköld, Fredrik","Adelsköld,-Fredrik","Adelsköld, Fredrik",,,Arken_only
3380,NaN,NaN,"Adelsköld, Sofia Marie","Adelsköld,-Sofia-Marie","Adelsköld, Sofia Marie",,,Arken_only
3381,NaN,NaN,"Adelswärd, Adolph Theodor","Adelswärd,-Adolph-Theodor","Adelswärd, Adolph Theodor",,,Arken_only


In [16]:
#mergewithLibris = Arken_only[Arken_only["Auktoritetspost"].notnull()].copy() 
ArkenOnlyWithAuthrec = Arken_only[Arken_only["Auktoritetspost"] != ''].copy() 
ArkenOnlyWithAuthrec.to_csv(r'ArkenOnlyWithAuthrec.csv')   
ArkenOnlyWithAuthrec

,WD,arkid,nameAuth,urlAuth,Auktoriserad,Datum,Auktoritetspost,WD_Arken
3376,NaN,NaN,Adams-Ray (släkt),Adams-Ray-släkt,Adams-Ray (släkt),,ediffah:kb:688055:1137069059,Arken_only
3378,NaN,NaN,Adelsköld (släkt),Adelsköld-släkt,Adelsköld (släkt),,ediffah:kb:482202:1457702780,Arken_only
3382,NaN,NaN,"Adlercreutz, Hedvig","Adlercreutz,-Hedvig","Adlercreutz, Hedvig",1832-1905,ediffah:kb:584691:1348770251,Arken_only
3390,NaN,NaN,"Aghnides, Thanassis","Aghnides,-Thanassis","Aghnides, Thanassis",1889-1984,ediffah:kb:285979:1415978596,Arken_only
3391,NaN,NaN,"Agrell, Carl Christian","Agrell,-Carl-Christian","Agrell, Carl Christian",1780-1838,ediffah:kb:587676:1189788900,Arken_only
3392,NaN,NaN,"Ahlbom, Frans Leonard","Ahlbom,-Frans-Leonard","Ahlbom, Frans Leonard",1816-1860,ediffah:kb:587676:1189788900,Arken_only
3393,NaN,NaN,"Ahlgren, Gerda","Ahlgren,-Gerda","Ahlgren, Gerda",1856-1945,ediffah:kb:671155:1436943082,Arken_only
3395,NaN,NaN,"Ahlström, Christine","Ahlström,-Christine","Ahlström, Christine",1807-1890,ediffah:kb:196469:1384421920,Arken_only
3396,NaN,NaN,"Ahnfelt-Rönne, Ragna","Ahnfelt-Rönne,-Ragna","Ahnfelt-Rönne, Ragna",1890-1951,ediffah:kb:963551:1415345577,Arken_only
3397,NaN,NaN,"Ahnlund, Mats",ahnlund-mats,"Ahnlund, Mats",1944-,http://libris.kb.se/resource/auth/306056,Arken_only


In [17]:
mergewithLibris
mergewithLibris.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2723 entries, 3372 to 6094
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   WD               0 non-null      object
 1   arkid            0 non-null      object
 2   nameAuth         2723 non-null   object
 3   urlAuth          2723 non-null   object
 4   Auktoriserad     2723 non-null   object
 5   Datum            2723 non-null   object
 6   Auktoritetspost  2723 non-null   object
 7   WD_Arken         2723 non-null   object
dtypes: object(8)
memory usage: 191.5+ KB


## Places

TBD

In [18]:
urlbase = "https://arken.kb.se/taxonomy/index/id/42?sort=alphabetic&sortDir=asc&page="
dfp = pd.DataFrame(columns=['nameAuth', 'urlAuth', 'Auktoriserad', 'Datum', 'Auktoritetspost'])
def check_Taxonomy(url):
    r_entry = http.request('GET', url)
    soup_entry = BeautifulSoup(r_entry.data, "html.parser")
    fields = soup_entry.select('div.field')
    for f in fields:
        h3 = f.select("h3")
        divText = f.select("div")
        if len(h3) > 0:
            if "Taxonomi" in h3[0].getText():
                print("\tTaxonomi: " + divText[0].getText().strip())
                Taxonomi = divText[0].getText().strip()
    return True
    
for i in range(1,10):
    url = urlbase + str(i)
    print(url)
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, "html.parser")
    for link in soup.select('table  a[href]'):
        nameAuth = link.string
        urlAuth = urllib.parse.unquote(link['href'].split("/")[1])
        print ("\t",urlAuth, nameAuth)    
        urlentry = urlbase_entry + link['href']
        print ("\t\t",urlentry)
        if check_Taxonomy(urlentry):
            print("True")
    

https://arken.kb.se/taxonomy/index/id/42?sort=alphabetic&sortDir=asc&page=1
	 abessinien-ca-990-1974 Abessinien (ca 990-1974)
		 https://arken.kb.se/abessinien-ca-990-1974
	Taxonomi: Platser
True
	 Afrika Afrika
		 https://arken.kb.se/Afrika
	Taxonomi: Platser
True
	 Aix-en-Provence Aix-en-Provence
		 https://arken.kb.se/Aix-en-Provence
	Taxonomi: Platser
True
	 Algeriet Algeriet
		 https://arken.kb.se/Algeriet
	Taxonomi: Platser
True
	 alingsas Alingsås
		 https://arken.kb.se/alingsas
	Taxonomi: Platser
True
	 Amerika Amerika
		 https://arken.kb.se/Amerika
	Taxonomi: Platser
True
	 Amiens Amiens
		 https://arken.kb.se/Amiens
	Taxonomi: Platser
True
	 Arboga Arboga
		 https://arken.kb.se/Arboga
	Taxonomi: Platser
True
	 argentina Argentina
		 https://arken.kb.se/argentina
	Taxonomi: Platser
True
	 arjeplog Arjeplog
		 https://arken.kb.se/arjeplog
	Taxonomi: Platser
True
	 arkangelsk Arkangelsk
		 https://arken.kb.se/arkangelsk
	Taxonomi: Platser
True
	 arvidsjaur Arvidsjaur
		 https://

	Taxonomi: Platser
True
	 Gottröra Gottröra
		 https://arken.kb.se/Gottr%C3%B6ra
	Taxonomi: Platser
True
	 granvag Granvåg
		 https://arken.kb.se/granvag
	Taxonomi: Platser
True
	 Grekland Grekland
		 https://arken.kb.se/Grekland
	Taxonomi: Platser
True
	 Grez Grez
		 https://arken.kb.se/Grez
	Taxonomi: Platser
True
	 grisbacka Grisbacka
		 https://arken.kb.se/grisbacka
	Taxonomi: Platser
True
	 grundsjo Grundsjö
		 https://arken.kb.se/grundsjo
	Taxonomi: Platser
True
	 grundsunda Grundsunda
		 https://arken.kb.se/grundsunda
	Taxonomi: Platser
True
	 grangesberg Grängesberg
		 https://arken.kb.se/grangesberg
	Taxonomi: Platser
True
	 Gräsmark Gräsmark
		 https://arken.kb.se/Gr%C3%A4smark
	Taxonomi: Platser
True
	 Guangdong Guangdong
		 https://arken.kb.se/Guangdong
	Taxonomi: Platser
True
	 gunnarn-tid-bastutrask Gunnarn (tid. Bastuträsk)
		 https://arken.kb.se/gunnarn-tid-bastutrask
	Taxonomi: Platser
True
	 Gällivare Gällivare
		 https://arken.kb.se/G%C3%A4llivare
	Taxonomi: Platser


	Taxonomi: Platser
True
	 Lindau Lindau
		 https://arken.kb.se/Lindau
	Taxonomi: Platser
True
	 Lindesberg Lindesberg
		 https://arken.kb.se/Lindesberg
	Taxonomi: Platser
True
	 Linköping Linköping
		 https://arken.kb.se/Link%C3%B6ping
	Taxonomi: Platser
True
	 litauen Litauen
		 https://arken.kb.se/litauen
	Taxonomi: Platser
True
	 ljusdal Ljusdal
		 https://arken.kb.se/ljusdal
	Taxonomi: Platser
True
	 Ljusnedal Ljusnedal
		 https://arken.kb.se/Ljusnedal
	Taxonomi: Platser
True
	 London London
		 https://arken.kb.se/London
	Taxonomi: Platser
True
	 long-island Long Island
		 https://arken.kb.se/long-island
	Taxonomi: Platser
True
	 Ludvika Ludvika
		 https://arken.kb.se/Ludvika
	Taxonomi: Platser
True
	 Luleå Luleå
		 https://arken.kb.se/Lule%C3%A5
	Taxonomi: Platser
True
	 Lund Lund
		 https://arken.kb.se/Lund
	Taxonomi: Platser
True
	 lycksele Lycksele
		 https://arken.kb.se/lycksele
	Taxonomi: Platser
True
	 Lysekil Lysekil
		 https://arken.kb.se/Lysekil
	Taxonomi: Platser
True
	 

	Taxonomi: Platser
True
	 san-francisco San Francisco
		 https://arken.kb.se/san-francisco
	Taxonomi: Platser
True
	 Sandhamn Sandhamn
		 https://arken.kb.se/Sandhamn
	Taxonomi: Platser
True
	 sandtrask Sandträsk
		 https://arken.kb.se/sandtrask
	Taxonomi: Platser
True
	 Sankt-Petersburg Sankt Petersburg
		 https://arken.kb.se/Sankt-Petersburg
	Taxonomi: Platser
True
	 sapmi Sapmi
		 https://arken.kb.se/sapmi
	Taxonomi: Platser
True
	 sardinien Sardinien
		 https://arken.kb.se/sardinien
	Taxonomi: Platser
True
	 Saskatchewan-Canada Saskatchewan
		 https://arken.kb.se/Saskatchewan-Canada
	Taxonomi: Platser
True
	 saudiarabien Saudiarabien
		 https://arken.kb.se/saudiarabien
	Taxonomi: Platser
True
	 Schweiz Schweiz
		 https://arken.kb.se/Schweiz
	Taxonomi: Platser
True
	 selanger Selånger
		 https://arken.kb.se/selanger
	Taxonomi: Platser
True
	 seskaro Seskarö
		 https://arken.kb.se/seskaro
	Taxonomi: Platser
True
	 Sibirien Sibirien
		 https://arken.kb.se/Sibirien
	Taxonomi: Platser
T

	Taxonomi: Platser
True
	 Uppland Uppland
		 https://arken.kb.se/Uppland
	Taxonomi: Platser
True
	 Uppsala Uppsala
		 https://arken.kb.se/Uppsala
	Taxonomi: Platser
True
	 Vadstena Vadstena
		 https://arken.kb.se/Vadstena
	Taxonomi: Platser
True
	 Vaksala Vaksala
		 https://arken.kb.se/Vaksala
	Taxonomi: Platser
True
	 Vasa Vasa
		 https://arken.kb.se/Vasa
	Taxonomi: Platser
True
	 Vaxholm Vaxholm
		 https://arken.kb.se/Vaxholm
	Taxonomi: Platser
True
	 Versailles Versailles
		 https://arken.kb.se/Versailles
	Taxonomi: Platser
True
	 vietnam Vietnam
		 https://arken.kb.se/vietnam
	Taxonomi: Platser
True
	 Vifolka-härad Vifolka härad
		 https://arken.kb.se/Vifolka-h%C3%A4rad
	Taxonomi: Platser
True
	 vilhelmina-2 Vilhelmina
		 https://arken.kb.se/vilhelmina-2
	Taxonomi: Platser
True
	 Vilnius Vilnius
		 https://arken.kb.se/Vilnius
	Taxonomi: Platser
True
	 vimmerby Vimmerby
		 https://arken.kb.se/vimmerby
	Taxonomi: Platser
True
	 vindeln Vindeln
		 https://arken.kb.se/vindeln
	Taxonomi